Install dan import library

In [ ]:
!pip install nlp-id 
!pip install Sastrawi
!pip install wordcloud
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 4.8 MB/s 
     |████████████████████████████████| 31.0 MB 1.5 MB/s 
     |████████████████████████████████| 1.5 MB 45.6 MB/s 
  Created wheel for nlp-id: filename=nlp_id-0.1.13.0-py3-none-any.whl size=8074943 sha256=49165a99e2c1fd108f4afb7f4ffb5cd3a8ba4a1ef75b6ef7eaf5569625b4455c
  Stored in directory: /root/.cache/pip/wheels/c9/40/1e/1f65670842151a87e008b5df647bf5d8b034e762e1934fd7f3
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449924 sha256=04d5ce7127117080856a69c9aed07d22625f70c21ba589122a2bb11ea10b1419
  Stored in directory: /root/.cache/pip/wheels/23/18/48/8fd6ec11da38406b309470566d6f099c04805d2ec61d7829e7
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=9818eddd0b09ddac9ed58feb368dcfcf23a2b10662c28b77fad003b564f30190
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

Load data dari Bu Yulian (train data)

In [ ]:
data_raw = pd.read_csv('https://raw.githubusercontent.com/kelvin2401/stki/main/datasetcoba.csv')
#data_raw = data_raw.loc[np.random.choice(data_raw.index, size=2000)]
data_raw.shape

(900, 8)

#Preprocessing

In [ ]:
missing_values_check = data_raw.isnull().sum()
print(missing_values_check)

Tweet     0
KS       35
KH       35
DH       35
TDH      35
SWA      35
LO       35
AK       36
dtype: int64


In [ ]:
data_raw_baru=data_raw.dropna(axis=0)

In [ ]:
categories = list(data_raw_baru.columns.values)
categories = categories[1:]
print(categories)

['KS', 'KH', 'DH', 'TDH', 'SWA', 'LO', 'AK']


In [ ]:
# Import kamus bahasa slang
slang_dict = pd.read_csv('https://raw.githubusercontent.com/kelvin2401/stki/main/slang.txt', delimiter = ";")
dict_slang = dict(slang_dict.values)
print (slang_dict)

          &                                          dan
0         +                                       tambah
1         /                                         atau
2     22nya                                   dua-duanya
3        3m  mencuci tangan memakai masker menjaga jarak
4       7an                                       tujuan
...     ...                                          ...
5592  yyaaa                                           ya
5593      z                                         saja
5594     za                                         saja
5595   zama                                        zaman
5596   zonk                                        bodoh

[5597 rows x 2 columns]


In [ ]:
# https://github.com/kumparan/nlp-id
from nlp_id.tokenizer import Tokenizer
from nlp_id.lemmatizer import Lemmatizer 

lemmatizer = Lemmatizer()
tokenizer = Tokenizer()

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
stop_factory = StopWordRemoverFactory()
more_stopword = ['dengan', 'ia','bahwa','oleh','aku','kamu','bukan','nya','iya','user','sih','jadi','deh','rt','kalau','apa','mau','memang']
data_stopword = stop_factory.get_stop_words() + more_stopword # menambah stopword

def preprocess(s):
    s = re.sub(r'(\\x[0-9a-fA-Z]{2})', '', s)
  #Remove URL
    s = re.sub(r'https?://\S+|www\.\S+', " ", s)
  #Remove Mentions
    s = re.sub(r'@\w+',' ',s)
  #Remove Punctuation
    s= re.sub(r'[^\w\s\d]',' ',s)
  #Remove Digits
    s = re.sub(r'\d+', ' ', s)
  #Remove HTML tags
    s = re.sub('r<.*?>',' ', s)
  #Remove Hastags
    s = re.sub(r'#\w+', ' ', s)
    s = re.sub(r'[^a-zA-Z]', ' ', s) # remove symbol dan angka
    hasil=[]
    word_token = tokenizer.tokenize(s) #tokenisaai
    for word in word_token:
        word = word.strip().lower()   #case folding
        if word in dict_slang:
            word= dict_slang[word]
        if word not in data_stopword: #stopword removal
            word = lemmatizer.lemmatize(word) #lemmatization
            hasil.append(word)
        else:
            continue
    result_sentence = " ".join(hasil).strip() #penggabungan kata hasil pre prosesing
    #print(result_sentence)
    return result_sentence

In [ ]:
#Melakukan Proses Pre Prosesing pada Kolom text Dataset 
result_preprocess = []
for i, row in tqdm(data_raw_baru.iterrows(), total=data_raw_baru.shape[0]):
  result_preprocess.append(preprocess(row['Tweet']))
'Done!'

100%|██████████| 864/864 [00:00<00:00, 2640.61it/s]


'Done!'

In [ ]:
#Menampilkan hasil pre prosesing pada kolom baru yaitu text clean
data_raw_baru['text_clean'] = result_preprocess

In [ ]:
data_raw_baru['text_clean']

0                    salah jokowi ahok kafir kafir ateis
1      biar biar tahi kena bapak loh kan suka sama ah...
2      prestasi ahok mjadi mafia koruptor nista agama...
3      ulama kompak nyata haram pimpin kafir pilih ah...
4      dik tinggi bodoh larang bawa agama biar orang ...
                             ...                        
892                                          wasit picek
893    orang kaya indonesia orang indonesia orang cin...
896           manusia berkelakuan cebong hina marah hina
897     media pura bego ajak bego orang nama bodoh parah
899                                            wong gila
Name: text_clean, Length: 864, dtype: object

In [ ]:
train = data_raw_baru
train_text = train['text_clean']

Load data manual labeling (test data)

In [ ]:
data_raw = pd.read_csv('https://raw.githubusercontent.com/muhammadariffaizin/sistem-temu-kembali-informasi/master/2_labelling/catatanali07_merged_labelled.csv')
data_raw.head()

,Unnamed: 0,Unnamed: 0.1,index,Id,Tweet Time,Url,Tweet Text,Likes Count,Reply Count,Retweet Count,...,User Following Count,User Likes Count,User Tweets Count,KS,KH,DH,TDH,SWA,LO,AK
0,0,0,182,1492310337122889729,2022-02-12 01:31:24+00:00,https://twitter.com/CatatanAli07/status/149231...,#FotoProfilBaru https://t.co/2nj1RH1qRV,0,0,0,...,1559.0,226.0,4473.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1208,1492328251255308290,2022-02-12 02:42:35+00:00,https://twitter.com/RheinaHerlina/status/14923...,@CatatanAli07 Ras🤸‍♀️🤸‍♀️,0,1,0,...,12813.0,121022.0,61084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1207,1492346287034757120,2022-02-12 03:54:15+00:00,https://twitter.com/didihadid_/status/14923462...,@CatatanAli07 Eh Sapri,0,1,0,...,1704.0,37228.0,159285.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,181,1492352188148219908,2022-02-12 04:17:42+00:00,https://twitter.com/CatatanAli07/status/149235...,JILAT TERUS \n\nISENG ISENG BERHADIAH YA DIE 😂...,1,0,1,...,1559.0,226.0,4473.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1206,1492380257420455937,2022-02-12 06:09:15+00:00,https://twitter.com/ArgentianaA/status/1492380...,@CatatanAli07 Ali banget itu 😆,0,1,0,...,2602.0,5488.0,291584.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Preprocessing test data

In [ ]:
data_raw = data_raw.dropna(how='all',subset=['KS','KH','DH','TDH','SWA','LO','AK'])

In [ ]:
data_raw

,Unnamed: 0,Unnamed: 0.1,index,Id,Tweet Time,Url,Tweet Text,Likes Count,Reply Count,Retweet Count,...,User Following Count,User Likes Count,User Tweets Count,KS,KH,DH,TDH,SWA,LO,AK
141,141,141,162,1513374464251428864,2022-04-11 04:32:44+00:00,https://twitter.com/CatatanAli07/status/151337...,"TODAY, \n\nThousands of people take part in a ...",81,5,50,...,1559.0,226.0,4473.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
151,151,151,160,1513389082495557632,2022-04-11 05:30:49+00:00,https://twitter.com/CatatanAli07/status/151338...,Yang kemarin juga dihajar cebong karna provoka...,11,2,2,...,1559.0,226.0,4473.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
179,179,179,156,1513479880931160064,2022-04-11 11:31:37+00:00,https://twitter.com/CatatanAli07/status/151347...,Muda doang NYALI kalah sama ADE ARMANDO \n\nPa...,84,14,17,...,1559.0,226.0,4473.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
303,303,303,140,1517035651870629888,2022-04-21 07:00:59+00:00,https://twitter.com/CatatanAli07/status/151703...,Alhamdulillah Puasa hari ini dapat Tambahan PA...,3,1,4,...,1559.0,226.0,4473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,306,306,139,1517062652434202624,2022-04-21 08:48:16+00:00,https://twitter.com/CatatanAli07/status/151706...,Gua Difitnah Dapat TANAH dari morotin Pacar\nG...,142,44,7,...,1559.0,226.0,4473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1177,1177,7,1545609653739958272,2022-07-09 03:23:52+00:00,https://twitter.com/CatatanAli07/status/154560...,Kan itu nama ko setan 🙄 https://t.co/gI4q8JOfgc,0,0,0,...,1559.0,226.0,4473.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1178,1178,1178,6,1545734677410893824,2022-07-09 11:40:40+00:00,https://twitter.com/CatatanAli07/status/154573...,Dulu pernah dapat GA PARFUM udah 2 Tahun gak d...,0,0,0,...,1559.0,226.0,4473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1180,1180,1180,5,1545836847682945024,2022-07-09 18:26:39+00:00,https://twitter.com/CatatanAli07/status/154583...,🤗,0,0,0,...,1559.0,226.0,4473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1201,1201,1201,3,1574365780224794624,2022-09-26 11:50:26+00:00,https://twitter.com/CatatanAli07/status/157436...,Jika PEMERINTAH ingin memberantas PERJUDIAN be...,5,5,3,...,1559.0,226.0,4473.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data_raw_baru=data_raw

In [ ]:
#Melakukan Proses Pre Prosesing pada Kolom text Dataset 
result_preprocess = []
for i, row in tqdm(data_raw_baru.iterrows(), total=data_raw_baru.shape[0]):
  result_preprocess.append(preprocess(row['Tweet Text']))
'Done!'

100%|██████████| 79/79 [00:00<00:00, 2494.17it/s]


'Done!'

In [ ]:
#Menampilkan hasil pre prosesing pada kolom baru yaitu text clean
data_raw_baru['text_clean'] = result_preprocess

In [ ]:
data_raw_baru['text_clean']

141     today thousands of people take part a demonstr...
151     kemarin hajar cebong provokasi yakin turun rum...
179          muda nyali kalah sama adek armando payah per
303     alhamdulillah puasa hari tambah pahala orang o...
306     fitnah tanah morotin pacar fitnah mokondo fitn...
                              ...                        
1177                                   kan nama kok setan
1178                 dulu pernah parfum tahun kirim kirim
1180                                                     
1201    perintah berantas judi berantas dulu sosial me...
1202                        henti lah kok main judi bodoh
Name: text_clean, Length: 79, dtype: object

In [ ]:
test_text = data_raw_baru['text_clean']

In [ ]:
y_test = data_raw_baru.drop(labels = ['Unnamed: 0','Id','Tweet Time','Url','Tweet Text','Likes Count','Reply Count','Retweet Count','Retweeted Tweet','User Id','User Name','User Location',
                             'text_clean','Unnamed: 0.1','index','User Follower Count','User Following Count','User Likes Count','User Tweets Count'], axis=1)

In [ ]:
y_test = y_test.fillna(0)
y_test

,KS,KH,DH,TDH,SWA,LO,AK
141,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151,0.0,0.0,1.0,0.0,0.0,0.0,0.0
179,0.0,0.0,0.0,1.0,0.0,0.0,0.0
303,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1177,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1178,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1180,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Feature Extraction (TFIDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [ ]:
x_train = vectorizer.transform(train_text)
x_test = vectorizer.transform(test_text)

In [ ]:
y_train = train.drop(labels = ['Tweet','text_clean'], axis=1)

#Klasifikasi

In [ ]:
from sklearn.ensemble import BaggingClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

bagClassifier = ClassifierChain(BaggingClassifier(n_jobs=-1))
bagClassifier.fit(x_train, y_train)
bagPreds = bagClassifier.predict(x_test)

#Evaluasi

In [ ]:
from sklearn.metrics import accuracy_score
label_names = ['Konteks Sosial', 'Konteks Historis', 'Dehumanisasi', 'Tuduhan Negatif', 'Serangan Pada Wanita', 'Mempertanyakan Grup', 'Ancaman Kelompok']
# accuracy
print("Accuracy = ",accuracy_score(y_test,bagPreds))
print("\n")
f1_score_macro = f1_score(y_test, bagPreds, average='macro', zero_division=0)
print('f1_score_macro : ',f1_score_macro)
print("\n")
f1_score_micro = f1_score(y_test, bagPreds, average='micro', zero_division=0)
print('f1_score_micro : ',f1_score_micro)
print("\n")
f1_score_weighted = f1_score(y_test, bagPreds, average='weighted', zero_division=0)
print('f1_score_weighted : ',f1_score_weighted)
print("\n")
print(classification_report(y_test, bagPreds,target_names=label_names, zero_division=0))
print("\n")
multilabel_confusion_matrix(y_test, bagPreds)

Accuracy =  0.0379746835443038


f1_score_macro :  0.12036206794991787


f1_score_micro :  0.2677165354330709


f1_score_weighted :  0.2560979520777019


                      precision    recall  f1-score   support

      Konteks Sosial       0.38      0.20      0.26        15
    Konteks Historis       0.00      0.00      0.00         2
        Dehumanisasi       0.24      0.44      0.31         9
     Tuduhan Negatif       0.17      0.77      0.27        13
Serangan Pada Wanita       0.00      0.00      0.00         0
 Mempertanyakan Grup       0.00      0.00      0.00         0
    Ancaman Kelompok       0.00      0.00      0.00         1

           micro avg       0.20      0.42      0.27        40
           macro avg       0.11      0.20      0.12        40
        weighted avg       0.25      0.42      0.26        40
         samples avg       0.18      0.12      0.14        40





array([[[59,  5],
        [12,  3]],

       [[77,  0],
        [ 2,  0]],

       [[57, 13],
        [ 5,  4]],

       [[16, 50],
        [ 3, 10]],

       [[79,  0],
        [ 0,  0]],

       [[79,  0],
        [ 0,  0]],

       [[76,  2],
        [ 1,  0]]])